# **1. Directiva Ordered**

Las secciones ordenadas resultan útiles para ordenar secuencialmente la salida del trabajo que se hace en paralelo.  El bloque estructurado que sigue a una directiva ordered se ejecuta en el orden en que las iteraciones se ejecutarían en un bucle secuencial. Una directiva ordered debe estar dentro de la extensión dinámica de una construcción for o parallel **for**. En la ejecución de una construcción for o parallel for con una cláusula ordered, las construcciones ordered se ejecutan estrictamente en el orden en que se ejecutarían en una ejecución secuencial del bucle.

Las restricciones a la directiva ordered son las siguientes:

* Una iteración de un bucle con una construcción for no debe ejecutar la misma directiva ordenada más de una vez y no debe ejecutar más de una directiva ordered.

El programa siguiente imprime los índices en un orden secuencial:


In [24]:
code = """
#include <omp.h>    // Cabecera OpenMP
#include <cstdio>

void work(int k)
{
    #pragma omp ordered
        printf(" %d", k);
}

int main() 
{

 /* #pragma omp parallel
  {*/
    int ub =10;
    int st=1;
    int lb=0;
    
    #pragma omp for ordered schedule(dynamic)
  
    
    for (int i=lb; i<ub; i+=st)
          work(i);

  //}

  return 0;
}

"""
text_file = open("ordered.cpp", "w")
text_file.write(code)
text_file.close()

In [26]:
!g++ -o ordered -fopenmp ordered.cpp

In [28]:
%env OMP_NUM_THREADS=7
!./ordered

env: OMP_NUM_THREADS=7
 0 1 2 3 4 5 6 7 8 9

# **2. Directiva Atomic**


En el ejemplo siguiente se evitan las condiciones de carrera, (actualizaciones simultáneas de un elemento de x por muchos subprocesos), al usar la directiva atómica:
La ventaja de usar la directiva atomic en este ejemplo es que permite que las actualizaciones de dos elementos distintos de x se produzcan en paralelo. Si, en su lugar, se usa una directiva crítica, todas las actualizaciones a elementos de x se ejecutan de forma serial, (aunque no en un orden asegurado).

La directiva atomic solo se aplica a la instrucción C o C++ inmediatamente a continuación de ella. Como resultado, los elementos de y no se actualizan atómicamente en este ejemplo

In [105]:
code="""
#include <omp.h>
#include <stdio.h>

//#define MAX 10000

int main() {
   int count = 0;
   #pragma omp parallel 
   {
     #pragma omp atomic
      count++;
   }
   printf("Number of threads: %d\\n", count);
}
"""
text_file = open("atomic.cpp", "w")
text_file.write(code)
text_file.close()

In [130]:
code = """
#include <omp.h>
#include <stdio.h>

int main()
{
    int sum = 0;

    #pragma omp parallel 
    {
        #pragma omp for
        for (int i = 0; i < 1000; i++) {
            #pragma omp atomic
            sum ++;
        }
    }

    printf("sum = %d\\n", sum);

    return 0;
}

"""
text_file = open("atomic.cpp", "w")
text_file.write(code)
text_file.close()

In [131]:
!g++ -o atomic -fopenmp atomic.cpp

In [134]:
%env OMP_NUM_THREADS=1000
!./atomic

env: OMP_NUM_THREADS=1000
sum = 1000


# **2. Directiva Locks (Bloqueos)**
En el ejemplo siguiente, el argumento de las funciones de bloqueo debería tener el tipo omp_lock_t y no es necesario vaciarlo. Las funciones de bloqueo hacen que los subprocesos estén inactivos mientras esperan la entrada de la primera sección crítica, pero para realizar otro trabajo mientras esperan la entrada al segundo. La función omp_set_lock bloquea, pero la función omp_test_lock no, lo que permite que se trabaje en skip().

In [6]:
code = """
#include <iostream>
#include <omp.h>
#include <stdlib.h>
#include <time.h>
#include <unistd.h>

int main() {
    srand(time(NULL)); // Inicializar semilla para generación de números aleatorios

    omp_lock_t lock; // Declarar lock de OpenMP
    omp_init_lock(&lock); // Inicializar lock

    #pragma omp parallel 
    {
        int id = omp_get_thread_num();
        int rand_sleep = rand() % 10 + 1; // Generar número aleatorio entre 1 y 10
        sleep(rand_sleep); // Esperar un tiempo aleatorio antes de acceder a la región crítica

        while(!omp_test_lock(&lock)) { // Mientras no pueda tomar el lock
            std::cout << "Thread " << id << " espera para tomar el lock" << std::endl;
            sleep(1); // Esperar un segundo antes de volver a intentar tomar el lock
        }

        std::cout << "Thread " << id << " está en la región crítica" << std::endl;
        for (int i = 1; i <= 10; i++) {
            std::cout << "Thread " << id << " imprime " << i << std::endl;
            sleep(1);
        }

        omp_unset_lock(&lock); // Liberar el lock
    }

    omp_destroy_lock(&lock); // Destruir el lock

    return 0;
}


"""
text_file = open("lock.cpp", "w")
text_file.write(code)
text_file.close()

In [7]:
!g++ -o lock -fopenmp lock.cpp

In [8]:
%env OMP_NUM_THREADS=10
!./lock

env: OMP_NUM_THREADS=10
Thread Thread 4 espera para tomar el lock
7 está en la región crítica
Thread 7 imprime 1
Thread 4 espera para tomar el lock
Thread 7 imprime 2
Thread 0 espera para tomar el lock
Thread 4 espera para tomar el lock
Thread 7 imprime 3
Thread 6 espera para tomar el lock
Thread 1 espera para tomar el lock
Thread 5Thread  espera para tomar el lock
Thread 3 espera para tomar el lock
2 espera para tomar el lock
Thread 0 espera para tomar el lock
Thread 4 espera para tomar el lock
Thread 7 imprime 4
Thread Thread 16 espera para tomar el lock espera para tomar el lock
Thread 0 espera para tomar el lock

Thread 2 espera para tomar el lock
Thread 5 espera para tomar el lock
Thread 3 espera para tomar el lock
Thread 4 espera para tomar el lock
Thread 7 imprime 5
Thread 1 espera para tomar el lock
Thread 5 espera para tomar el lock
Thread 2 espera para tomar el lock
Thread 6 espera para tomar el lock
Thread Thread 0 espera para tomar el lock
3 espera para tomar el lock
Thread